In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

from sklearn.linear_model import LinearRegression

In [2]:
input_fn = './propagation_inputs.csv'

model = 'gpt2'
scores_fn = './70M/scores:dirty_me_model.csv'

In [3]:
input_df = pd.read_csv(input_fn, header=[0])
input_df.head(1)

,example_index,text,sub_index,original,synonym,substituted?
0,159769,New stationery store has Inglewood abuzz\n\nPr...,924,nice,good,True


In [4]:
score_df = pd.read_csv(scores_fn, header=None)
score_df.columns = ['example_idx', 'w1_prob', 'w2_prob', 'w1_rank', 'w2_rank']
df = input_df.join(score_df)
df.tail(1)

,example_index,text,sub_index,original,synonym,substituted?,example_idx,w1_prob,w2_prob,w1_rank,w2_rank
44999,512544,Crew members of the 48th Fight Wing prepare an...,952,team,group,True,512544,0.041534,0.007683,2,17


In [5]:
all(df.example_index == df.example_idx)

True

In [6]:
df[['original', 'substituted?']].groupby('original').count().head(5)

,substituted?
original,
accept,1000
analysis,1000
area,1000
big,1000
business,1000


In [7]:
df[['original', 'substituted?']].groupby('original').mean().head(5)

,substituted?
original,
accept,0.520
analysis,0.510
area,0.494
big,0.497
business,0.513


In [18]:
linear_adj = True

In [19]:
if linear_adj:
    logistic_models = {}
    for i, g in df.groupby("original"):
        temp_model = LinearRegression().fit(np.log(np.array(g["w2_prob"]+1e-8)).reshape(-1, 1), np.log(g["w1_prob"]+1e-8))
        logistic_models[i] = temp_model

    def func(row):
        x = np.array(row["w2_prob"]).reshape(1, -1)
        prediction = np.exp(logistic_models[row["original"]].predict(np.log(x+1e-8))[0])
        return prediction > row["w1_prob"]

    df['prediction'] = df.apply(func, axis=1)
    df["prediction"] = df.apply(func, axis=1)
else:
    df['prediction'] = df.w1_rank > df.w2_rank

In [20]:
preds = df[['original', 'prediction']].groupby('original').mean()

In [21]:
df['correctness'] = df['prediction'] == df['substituted?']
len(df), df['correctness'].mean()

(45000, 0.5008444444444444)

In [22]:
corrects = df[['original', 'correctness']].groupby('original').mean().sort_values('correctness')

In [23]:
preds.join(corrects).sort_values('correctness')

,prediction,correctness
original,,
totally,0.512,0.464
more,0.516,0.472
own,0.429,0.475
big,0.520,0.477
monitor,0.503,0.479
return,0.475,0.481
first,0.518,0.481
provide,0.496,0.482
equal,0.512,0.485
